<a href="https://colab.research.google.com/github/ricardomsanti/douExplore/blob/master/DEX_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PACKAGES

In [1]:

!pip install html5lib
!pip install pyspark
!pip install PyPDF2
!pip install html5lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=4e06ddc58402f76bd09373b540b567b00d026da9c4937ec11852fe3e064f26dc
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# DRIVE MOUNT

In [15]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# IMPORTS

In [3]:
! pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 7.3 MB/s eta 0:00:00


In [4]:
import os
from PyPDF2 import PdfReader
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# SPARK


In [8]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("my_app") \
    .getOrCreate()

# DOU - UNIÃO

In [1]:

FED_PATH = "/content/drive/MyDrive/DATA/DOU/SHARE"


In [10]:
import string
from string import punctuation
import pyspark.sql.functions as F

In [12]:

FILES_PATH = "/content/drive/MyDrive/DATA/DOU/SHARE"

FILES = [[x.name, \
          x.path]\
           for x in os.scandir(FILES_PATH) if ".pdf" in str(x)]

display(f"Total number of file: {len(FILES)}")

'Total number of file: 3'

In [14]:
text_lines = []
#ARQUIVOS
for f in FILES:
  obj = PdfReader(open(f[1], 'rb'))
  ext_line = []
  #PAGINAS
  for page in range(0, len(obj.pages)):
    main_text = obj.pages[page].extract_text().split("\n")
    #LINHAS
    for line in range(0,len(main_text)):
      ext_line = []
      ext_line = [f[1], page, line, main_text[line]]
      print(f"Page >> {page}  \t" +
      f" Line >> {line}  \t" +
      f" Object Pages: >> {len(obj.pages)}")
      text_lines.append(ext_line)

A saída de streaming foi truncada nas últimas 5000 linhas.
Page >> 242  	 Line >> 36  	 Object Pages: >> 267
Page >> 242  	 Line >> 37  	 Object Pages: >> 267
Page >> 242  	 Line >> 38  	 Object Pages: >> 267
Page >> 242  	 Line >> 39  	 Object Pages: >> 267
Page >> 242  	 Line >> 40  	 Object Pages: >> 267
Page >> 242  	 Line >> 41  	 Object Pages: >> 267
Page >> 242  	 Line >> 42  	 Object Pages: >> 267
Page >> 242  	 Line >> 43  	 Object Pages: >> 267
Page >> 242  	 Line >> 44  	 Object Pages: >> 267
Page >> 242  	 Line >> 45  	 Object Pages: >> 267
Page >> 242  	 Line >> 46  	 Object Pages: >> 267
Page >> 242  	 Line >> 47  	 Object Pages: >> 267
Page >> 242  	 Line >> 48  	 Object Pages: >> 267
Page >> 242  	 Line >> 49  	 Object Pages: >> 267
Page >> 242  	 Line >> 50  	 Object Pages: >> 267
Page >> 242  	 Line >> 51  	 Object Pages: >> 267
Page >> 242  	 Line >> 52  	 Object Pages: >> 267
Page >> 242  	 Line >> 53  	 Object Pages: >> 267
Page >> 242  	 Line >> 54  	 Object Pages

In [35]:

lines = pd.DataFrame(text_lines)
lines.columns = ["path", "pg_num","line", "string"]
data = spark.createDataFrame(lines)
data = data.withColumn("texto_regex", F.regexp_replace("string", "[\$#,\"!%&'()*+-./:;<=>?@^_`´{|}~\\\\]", ""))
data = data.withColumn("texto_trim", F.trim(data.texto_regex))
data = data.withColumn("trim_list", F.split("texto_trim", " "))
data.show(10, False)

+---------------------------------------------------------------------+------+----+-------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+
|path                                                                 |pg_num|line|string                                                                                                                               |texto_regex                                                                                                                    |texto_trim                         

In [64]:
from google.colab import files

data5 = data.toPandas()
data5.to_csv('data5.csv')
files.download('data5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
data6 = data.toPandas()
data6.to_parquet('data6.parquet')
files.download('data6.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
data2 = data.select("pg_num", "line", "trim_list")\
            .withColumn("exp_line", F.explode("trim_list"))
data2.show(10)

+------+----+--------------------+----------+
|pg_num|line|           trim_list|  exp_line|
+------+----+--------------------+----------+
|     0|   0|[REPÚBLICA, FEDER...| REPÚBLICA|
|     0|   0|[REPÚBLICA, FEDER...|FEDERATIVA|
|     0|   0|[REPÚBLICA, FEDER...|        DO|
|     0|   0|[REPÚBLICA, FEDER...|    BRASIL|
|     0|   0|[REPÚBLICA, FEDER...|         •|
|     0|   0|[REPÚBLICA, FEDER...|  IMPRENSA|
|     0|   0|[REPÚBLICA, FEDER...|  NACIONAL|
|     0|   1|[Ano, LXIV, Nº, 1...|       Ano|
|     0|   1|[Ano, LXIV, Nº, 1...|      LXIV|
|     0|   1|[Ano, LXIV, Nº, 1...|        Nº|
+------+----+--------------------+----------+
only showing top 10 rows



In [69]:
data7 = data.withColumn("exp_line", F.explode("trim_list"))
data7= data7.withColumn("IDX", F.monotonically_increasing_id())
data7 = data7.toPandas()

data7.to_csv("data7.csv")
files.download("data7.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
data3 = data2.select("exp_line")
data4 = data3.drop_duplicates()
data4 = data4.withColumn("IDX", F.monotonically_increasing_id())

53110